In [5]:
from keras.applications import MobileNet

In [9]:
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import fashion_mnist

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

X_train = np.array([cv2.resize(im, (128, 128)) for im in X_train])
X_test = np.array([cv2.resize(im, (128, 128)) for im in X_test])

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255.0
X_test /= 255.0

X_train = np.stack((X_train,)*3, axis=-1)
X_test = np.stack((X_test,)*3, axis=-1)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [17]:
X_train.shape
y_train.shape

(60000, 10)

In [21]:
from keras import Model
from keras.layers import Dense

base_model = MobileNet(include_top=False, pooling='avg', weights='imagenet', input_shape=(128, 128, 3))
base_model.trainable = False

# Add new layers for Fashion MNIST classification
x = base_model.output
x = Dense(128, activation='relu')(x)
x = Dense(10, activation='softmax')(x)

# Create a new model
model = Model(inputs=base_model.input, outputs=x)

In [23]:
from keras.optimizers import Adam

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
batch_size = 32
epochs = 10

history = model.fit(X_train, y_train, batch_size=batch_size, epochs = epochs)

Epoch 1/10
1875/1875 [==============================] - 351s 187ms/step - loss: 0.2504 - accuracy: 0.9068
Epoch 2/10
1875/1875 [==============================] - 348s 186ms/step - loss: 0.2210 - accuracy: 0.9188
Epoch 3/10
1875/1875 [==============================] - 351s 187ms/step - loss: 0.1993 - accuracy: 0.9263
Epoch 4/10
1875/1875 [==============================] - 363s 194ms/step - loss: 0.1817 - accuracy: 0.9322
Epoch 5/10
1875/1875 [==============================] - 370s 197ms/step - loss: 0.1657 - accuracy: 0.9373
Epoch 6/10
1875/1875 [==============================] - 344s 184ms/step - loss: 0.1523 - accuracy: 0.9423
Epoch 7/10
1875/1875 [==============================] - 349s 186ms/step - loss: 0.1398 - accuracy: 0.9471
Epoch 8/10
1875/1875 [==============================] - 342s 182ms/step - loss: 0.1286 - accuracy: 0.9511
Epoch 9/10
1875/1875 [==============================] - 343s 183ms/step - loss: 0.1185 - accuracy: 0.9557
Epoch 10/10
1875/1875 [=======================

In [26]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')

313/313 - 62s - loss: 0.3429 - accuracy: 0.9062 - 62s/epoch - 199ms/step

Test accuracy: 0.9061999917030334


In [27]:
model.save('NN/model_fashion_mnist_mobilenet.h5')